In [1]:
import pdfplumber
import pandas as pd
import re

In [7]:
file_path = "static/files/demo2.pdf"
with pdfplumber.open(file_path) as pdf:
    table = pdf.pages[0].extract_table()

In [ ]:
table

In [5]:
data_frame = pd.DataFrame(table)
data_frame.columns = data_frame.iloc[0]
data_frame = data_frame.drop(0)

In [120]:
data_frame.columns = data_frame.columns.str.replace('\n', ' ')
required_columns = ['LINE', 'SKU', 'DESCRIPTION LINE ITEM COMMENTS', 'UNIT COST/ RETAIL PRICE', 'QTY', 'ITEM TOTAL']
required_columns = list(map(lambda text: re.sub(r'\s+', ' ', text.strip()), required_columns))
selected_df = data_frame[required_columns].copy()

selected_df = selected_df.iloc[:-1]
selected_df = selected_df.replace('\n',' ', regex=True)
final_df = selected_df.set_index("LINE")

In [121]:
final_df

,SKU,DESCRIPTION LINE ITEM COMMENTS,UNIT COST/ RETAIL PRICE,QTY,ITEM TOTAL
LINE,,,,,
1,1494819,Item Detail: INNO FOODS ALMOND FRUIT CRUNCH 90...,Unit Price: 13.19,6720.0,"88,636.80"
2,1552820,Item Detail: INNO ORGANIC GRAIN FREE KETO GRAN...,Unit Price: 244.12,576.0,"140,613.12"


In [122]:
json_output = final_df.to_json(orient = 'columns')
json_output

'{"SKU":{"1":"1494819","2":"1552820"},"DESCRIPTION LINE ITEM COMMENTS":{"1":"Item Detail: INNO FOODS ALMOND FRUIT CRUNCH 900G P280","2":"Item Detail: INNO ORGANIC GRAIN FREE KETO GRANOLA 600G P272"},"UNIT COST\\/ RETAIL PRICE":{"1":"Unit Price: 13.19","2":"Unit Price: 244.12"},"QTY":{"1":"6720.0","2":"576.0"},"ITEM TOTAL":{"1":"88,636.80","2":"140,613.12"}}'

In [99]:
selected_df.to_csv("output.csv")

In [128]:
ls = "SKU, ITEM TOTAL, DESCRIPTION LINE ITEM COMMENTS"

ans = ls.split(",")
ans = list(map(lambda text: re.sub(r'\s+', ' ', text.strip()), ans))
ans

['SKU', 'ITEM TOTAL', 'DESCRIPTION LINE ITEM COMMENTS']